In [1]:
pip install transformers torch

   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/9.7 MB 8.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.7 MB 5.9 MB/s eta 0:00:02
   ------------- -------------------------- 3.1/9.7 MB 5.8 MB/s eta 0:00:02
   ------------------ --------------------- 4.5/9.7 MB 5.8 MB/s eta 0:00:01
   ----------------------- ---------------- 5.8/9.7 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.7 MB 6.2 MB/s eta 0:00:01
   ----------------------------------- ---- 8.7/9.7 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/203.0 MB 6.7 MB/s eta 0:00:31
    --------------------------------------- 2.9/203.0 MB 7.3 MB/s eta 0:00:28
    --------------------------------------- 4.7/203.0 MB 7.5 MB/s eta 0:00:27
   - -------------

In [4]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Model ve Tokenizer yükleme
model_name = "t5-small"  # Daha güçlü bir model için "t5-large" kullanılabilir
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Özetlenecek metin
text = """
Hugging Face Transformers, NLP modellerini kullanmayı kolaylaştıran bir kütüphanedir.
BERT, T5 ve GPT gibi önceden eğitilmiş modelleri içerir. 
Bu kütüphane, araştırmacılar ve geliştiriciler için oldukça kullanışlıdır.
"""

# Özetleme için giriş metni hazırlama
input_text = "summarize: " + text
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# Özetleme
outputs = model.generate(input_ids, max_length=50, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Orijinal Metin:")
print(text)
print("\nÖzet:")
print(summary)


Orijinal Metin:

Hugging Face Transformers, NLP modellerini kullanmayı kolaylaştıran bir kütüphanedir.
BERT, T5 ve GPT gibi önceden eğitilmiş modelleri içerir. 
Bu kütüphane, araştırmacılar ve geliştiriciler için oldukça kullanışlıdır.


Özet:
kütüphane, araştrmaclar ve geliştiriciler için oldukça kullanşldr.


In [21]:
pip install rouge-score


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=a1dfef931bbcc9ff55b776f1c0a58aba2ed63d74df78cb0268acb71d507374f6
  Stored in directory: c:\users\msi-nb\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
Note: you may need to restart the kernel to use updated packages.


In [41]:
!pip install ipython




In [43]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from rouge_score import rouge_scorer
import os
import time
import psutil

# Symlink uyarısını devre dışı bırak
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

def calculate_rouge(reference, summary):
    """ROUGE skorlarını hesaplar."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, summary)
    return scores

def summarize_text(model_name="t5-small", text=""):
    """Metni özetler ve ölçümleri yapar."""
    # Model ve Tokenizer yükleme
    tokenizer = T5Tokenizer.from_pretrained(model_name, legacy=False)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Özetleme için giriş metni hazırlama
    input_text = "summarize: " + text
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    # Performans ölçümleri için bellek bilgisi
    process = psutil.Process()
    memory_before = process.memory_info().rss  # Byte cinsinden bellek kullanımı

    # Özetleme başlat
    start_time = time.time()
    outputs = model.generate(input_ids, max_length=200, min_length=60, length_penalty=1.0, num_beams=8, early_stopping=True)
    end_time = time.time()

    # Performans ölçümleri
    elapsed_time = end_time - start_time
    memory_after = process.memory_info().rss
    memory_usage = (memory_after - memory_before) / (1024 * 1024)  # MB cinsinden bellek artışı

    # Sonucu çözümle
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

    return summary, elapsed_time, memory_usage, model_name

# Kullanıcıdan metin al
print("T5 Modelini Seçin (t5-small, t5-large, t5-3b): ")
selected_model = input("Model (Varsayılan: t5-small): ") or "t5-small"

print("\nÖzetlenecek metni girin (Türkçe veya İngilizce): ")
input_text = input("Metin: ")

if input_text.strip():
    # Özetleme yap
    summary, elapsed_time, memory_usage, model_used = summarize_text(model_name=selected_model, text=input_text)

    # ROUGE hesaplama
    rouge_scores = calculate_rouge(input_text, summary)

    print("\nOrijinal Metin:")
    print(input_text)
    print("\nÖzet:")
    print(summary)
    print(f"\nModel: {model_used}")
    print(f"İşlem Süresi: {elapsed_time:.2f} saniye")
    print(f"Hafıza Kullanımı: {memory_usage:.2f} MB")
    print("\nROUGE Skorları:")
    print(f"ROUGE-1: {rouge_scores['rouge1']}")
    print(f"ROUGE-2: {rouge_scores['rouge2']}")
    print(f"ROUGE-L: {rouge_scores['rougeL']}")

    # Sonucu kaydet
    with open("summary_report.txt", "w", encoding="utf-8") as f:
        f.write(f"Model: {model_used}\n")
        f.write(f"İşlem Süresi: {elapsed_time:.2f} saniye\n")
        f.write(f"Hafıza Kullanımı: {memory_usage:.2f} MB\n")
        f.write("Orijinal Metin:\n")
        f.write(input_text + "\n\n")
        f.write("Özet:\n")
        f.write(summary + "\n\n")
        f.write("ROUGE Skorları:\n")
        f.write(f"ROUGE-1: {rouge_scores['rouge1']}\n")
        f.write(f"ROUGE-2: {rouge_scores['rouge2']}\n")
        f.write(f"ROUGE-L: {rouge_scores['rougeL']}\n")
    print("\nSonuç 'summary_report.txt' dosyasına kaydedildi.")
else:
    print("Geçerli bir metin girmediniz.")


T5 Modelini Seçin (t5-small, t5-large, t5-3b): 


Model (Varsayılan: t5-small):  t5-large



Özetlenecek metni girin (Türkçe veya İngilizce): 


Metin:  Para, mal ve hizmetlerin değiş-tokuşu için kullanılan araçlardan en yaygın olanı.[1] Para sözcüğü ile genellikle madenî para ve banknotlar kastedilmekle birlikte; ekonomide, vadesiz mevduatlar ve kredi kartları da parayı meydana getiren unsurlardan sayılır.[1] Vadeli mevduat, devlet tahvili gibi değişim araçları ise para benzeri olarak değerlendirilir.[1]  Para değer denkliğinin bir göstergesi olarak değer ölçütü, değişim ve saklama aracı olarak kullanılır.[2] Bu gösterge maddi ya da nominal değerde karşılıklar bulabilir. Maddî nitelikte paraya örnek olarak madeni para, banka teminat belgeleri ya da banknot, çek ya da senetler örnek verilebilir. Nominal nitelikte paraya ise banka hesabındaki para ya da kredi onayı örnek verilebilir. Para gündelik yaşamda takas aracı olarak kullanılır. Doğrudan doğruya takas yapan kişilerin ihtiyaçlarını karşılamasının yanında diğer takaslar için geçerli olması, parayı diğer takas araçlarından ayıran önemli bir özelliktir.  Günümüzde kullanılan 


Orijinal Metin:
Para, mal ve hizmetlerin değiş-tokuşu için kullanılan araçlardan en yaygın olanı.[1] Para sözcüğü ile genellikle madenî para ve banknotlar kastedilmekle birlikte; ekonomide, vadesiz mevduatlar ve kredi kartları da parayı meydana getiren unsurlardan sayılır.[1] Vadeli mevduat, devlet tahvili gibi değişim araçları ise para benzeri olarak değerlendirilir.[1]  Para değer denkliğinin bir göstergesi olarak değer ölçütü, değişim ve saklama aracı olarak kullanılır.[2] Bu gösterge maddi ya da nominal değerde karşılıklar bulabilir. Maddî nitelikte paraya örnek olarak madeni para, banka teminat belgeleri ya da banknot, çek ya da senetler örnek verilebilir. Nominal nitelikte paraya ise banka hesabındaki para ya da kredi onayı örnek verilebilir. Para gündelik yaşamda takas aracı olarak kullanılır. Doğrudan doğruya takas yapan kişilerin ihtiyaçlarını karşılamasının yanında diğer takaslar için geçerli olması, parayı diğer takas araçlarından ayıran önemli bir özelliktir.  Günümüzde ku